In [1]:
import numpy as np

from tvb.basic.profile import TvbProfile
TvbProfile.set_profile(TvbProfile.LIBRARY_PROFILE)

from tvb_nest.config import *
config = Config()
config.figures.SHOW_FLAG = True 
config.figures.SAVE_FLAG = True
config.figures.FIG_FORMAT = 'png'


2020-01-09 11:44:27,896 - INFO - numexpr.utils - NumExpr defaulting to 6 threads.


/home/docker/packages/tvb-root/scientific_library/tvb/datatypes/surfaces.py:61: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [2]:
# import nest
# nest.Models()
# from pynestml.frontend.pynestml_frontend import install_nest  # to_nest, 
# import os
# nest_path = os.environ["NEST_INSTALL_DIR"]
# modules_path = os.environ["MYMODULES_BLD_DIR"]
# modules_and_models = {"iaf_cond_deco2014": ["iaf_cond_deco2014"], 
# #                       "iaf_cond_nmda_deco2014": ["iaf_cond_nmda_deco2014"], 
# #                       "iaf_cond_ampa_gaba_deco2014": ["iaf_cond_ampa_gaba_deco2014"], 
# #                       "scale_connection": ["scale_connection"], 
#                       "tvb_rate_wongwang": ["tvb_rate_ampa_gaba_wongwang", "tvb_rate_nmda_wongwang", 
#                                             "tvb_rate_redwongwang_exc", "tvb_rate_redwongwang_inh"]
#                       }
# for module, models in modules_and_models.items():
#     models_path = os.path.join(modules_path, '%s' % module)
#     # Compile models
#     install_nest(models_path, nest_path)
#     module_name = module + "module"
#     # Intall into NEST
#     nest.Install(module_name)
#     for model in models:  
#         print("\n" + model + "\n")
#         try:
#             print(nest.GetDefaults(model))
#             n = nest.Create(model, 1)
#             status=nest.GetStatus(n)
#             print("\n" + model + "\n")
#             print(status)
#         except:
#             n1 = nest.Create("iaf_cond_ampa_gaba_deco2014", 1)
#             n2 = nest.Create("iaf_cond_nmda_deco2014", 1)
#             nest.Connect(n1, n2, 
#                          syn_spec={'model': "scale_connection", "receptor_type": 3, "weight": 1.0})
#             nest.Connect(n2, n1, 
#                          syn_spec={'model': "scale_connection", "receptor_type": 2, "weight": 1.0})
#             print("\n" + str(nest.GetStatus(nest.GetConnections(n1))))
#             print("\n" + str(nest.GetStatus(nest.GetConnections(n2))))

In [3]:
# from tvb_nest.nest_models.builders.nest_factory import compile_modules
# compile_modules(["iaf_cond_deco2014", "tvb_rate_wongwang" 
# # "iaf_cond_nmda_deco2014", "iaf_cond_ampa_gaba_deco2014", "scale_connection", "tvb_rate_wongwang"
#                 ])
# nest.Models()

In [4]:
# import pyNN.nest as sim; sim.setup(); sim.end(); print(sim)

In [5]:
from tvb_nest.config import CONFIGURED
from tvb.datatypes.connectivity import Connectivity
# Select the regions for the fine scale modeling with NEST spiking networks
nest_nodes_ids = []  # the indices of fine scale regions modeled with NEST
# In this example, we model parahippocampal cortices (left and right) with NEST
connectivity = Connectivity.from_file(config.DEFAULT_CONNECTIVITY_ZIP)
for id in range(connectivity.region_labels.shape[0]):
    if connectivity.region_labels[id].find("hippo") > 0:
        nest_nodes_ids.append(id)

In [6]:
# For interactive plotting:
# %matplotlib notebook  

# Otherwise:
%matplotlib inline 

from tvb_nest.config import CONFIGURED
from tvb_nest.examples.example import main_example
from tvb_nest.nest_models.builders.models.default_exc_io_inh_i import DefaultExcIOInhIBuilder
from tvb_nest.nest_models.builders.models.red_ww_exc_io_inh_i import RedWWExcIOInhIBuilder
from tvb_nest.nest_models.builders.models.ww_deco2014 import WWDeco2014Builder
from tvb_nest.interfaces.builders.models.red_ww_exc_io_inh_i \
    import RedWWexcIOinhIBuilder as InterfaceRedWWexcIOinhIBuilder
from tvb_nest.interfaces.builders.models.ww_deco2014 \
    import WWDeco2014Builder as InterfaceWWDeco2014Builder
from tvb_nest.interfaces.builders.models.wilson_cowan import \
    WilsonCowanBuilder as InterfaceWilsonCowanBuilder
from tvb_nest.interfaces.builders.models.generic_2d_oscillator import \
    Generic2DOscillatorBuilder as InterfaceGeneric2DOscillatorBuilder
from tvb_multiscale.plot.plotter import Plotter

from tvb.datatypes.connectivity import Connectivity
from tvb.simulator.simulator import Simulator
from tvb.simulator.models.reduced_wong_wang_exc_io_inh_i import ReducedWongWangExcIOInhI
from tvb.simulator.models.wilson_cowan_constraint import WilsonCowan
from tvb.simulator.models.generic_2d_oscillator_multiscale import Generic2dOscillator
from tvb.simulator.monitors import Raw  # , Bold  # , EEG

# # -----------------------------Generic2dOscillator oscillatory regime-----------------------------------------------
# model = Generic2dOscillator()
# model.d = np.array([0.1])
# model.e = np.array([0.0])
# model.f = np.array([1.0])
# model.g = np.array([1.0])
# model.I = np.array([0.0])
# model.tau = np.array([1.0])
# model.alpha = np.array([1.0])
# model.beta = np.array([0.0])
# model.a = np.array([0.0])
# model.b = np.array([-1.0])
# model.c = np.array([0.0])
# model.variables_of_interest = ["V", "W"]
# # -----------------------------------Wilson Cowan oscillatory regime------------------------------------------------
model = WilsonCowan()
model.tau_e = np.array([8.0])
model.tau_i = np.array([8.0])
model.c_ee = np.array([16.0])
model.c_ei = np.array([12.0])
model.c_ie = np.array([15.0])
model.c_ii = np.array([3.0])
model.a_e = np.array([1.3])
model.a_i = np.array([2.0])
model.b_e = np.array([4.0])
model.b_i = np.array([3.7])
model.P = np.array([0.0])
model.variables_of_interest = ["E", "I"]
main_example(model, DefaultExcIOInhIBuilder, InterfaceWilsonCowanBuilder,
             nest_nodes_ids, nest_populations_order=100, connectivity=connectivity, 
             simulation_length=100.0, tvb_state_variable_type_label="State Variables",
             exclusive_nodes=True, noise_strength=0.001, dt=0.1, config=CONFIGURED)

2020-01-09 11:44:28,821 - DEBUG - matplotlib - $HOME=/home/docker
2020-01-09 11:44:28,822 - DEBUG - matplotlib - CONFIGDIR=/home/docker/.config/matplotlib
2020-01-09 11:44:28,825 - DEBUG - matplotlib - matplotlib data path: /usr/share/matplotlib/mpl-data
2020-01-09 11:44:28,826 - DEBUG - matplotlib - loaded rc file /etc/matplotlibrc
2020-01-09 11:44:28,829 - DEBUG - matplotlib - matplotlib version 3.0.2
2020-01-09 11:44:28,830 - DEBUG - matplotlib - interactive is False
2020-01-09 11:44:28,831 - DEBUG - matplotlib - platform is linux
2020-01-09 11:44:28,832 - DEBUG - matplotlib - loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_colle

2020-01-09 11:44:28,875 - DEBUG - matplotlib - CACHEDIR=/home/docker/.cache/matplotlib
2020-01-09 11:44:28,877 - DEBUG - matplotlib.font_manager - Using fontManager instance from /home/docker/.cache/matplotlib/fontlist-v300.json
2020-01-09 11:44:28,986 - DEBUG - matplotlib.pyplot - Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-01-09 11:44:28,989 - DEBUG - matplotlib.pyplot - Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-01-09 11:44:29,598 - WARNING - tvb.traits - Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.datatypes.time_series.TimeSeries.labels_dimensions = Attr(field_type=<class 'dict'>, default={}, required=True)
2020-01-09 11:44:29,603 - WARNING - tvb.simulator.common - psutil module not available: no warnings will be issued when a
    simulation may require more memory than available
2020-01-09 11:44:29,605 - WARNING - tvb.traits - Field seems mutable and has

2020-01-09 11:44:32,636 - DEBUG - matplotlib.colorbar - Using auto colorbar locator on colorbar
2020-01-09 11:44:32,637 - DEBUG - matplotlib.colorbar - locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f467e9d5cc0>
2020-01-09 11:44:32,637 - DEBUG - matplotlib.colorbar - Setting pcolormesh
2020-01-09 11:44:32,656 - DEBUG - matplotlib.axes._base - update_title_pos
2020-01-09 11:44:32,933 - DEBUG - matplotlib.axes._base - update_title_pos
2020-01-09 11:44:33,048 - DEBUG - matplotlib.axes._base - update_title_pos
2020-01-09 11:44:33,323 - DEBUG - matplotlib.axes._base - update_title_pos
Building NEST network...
2020-01-09 11:44:33,337 - INFO - tvb_nest.nest_models.builders.base - Loading a NEST instance...
2020-01-09 11:44:33,337 - INFO - tvb_nest.nest_models.builders.base - Loading a NEST instance...
2020-01-09 11:44:33,340 - INFO - tvb_nest.nest_models.builders.base - NEST_INSTALL_DIR: /home/docker/env/neurosci/nest_build
2020-01-09 11:44:33,340 - INFO - tvb_nest.nest_model

2020-01-09 11:44:34,236 - INFO - tvb_multiscale.spiking_models.devices - <class 'tvb_multiscale.spiking_models.devices.DeviceSet'> of model spike_detector for I created!
2020-01-09 11:44:34,251 - INFO - tvb_multiscale.spiking_models.devices - <class 'tvb_multiscale.spiking_models.devices.DeviceSet'> of model multimeter for Excitatory created!
2020-01-09 11:44:34,251 - INFO - tvb_multiscale.spiking_models.devices - <class 'tvb_multiscale.spiking_models.devices.DeviceSet'> of model multimeter for Excitatory created!
2020-01-09 11:44:34,260 - INFO - tvb_multiscale.spiking_models.devices - <class 'tvb_multiscale.spiking_models.devices.DeviceSet'> of model multimeter for Inhibitory created!
2020-01-09 11:44:34,260 - INFO - tvb_multiscale.spiking_models.devices - <class 'tvb_multiscale.spiking_models.devices.DeviceSet'> of model multimeter for Inhibitory created!
2020-01-09 11:44:34,270 - INFO - tvb_multiscale.spiking_models.network - <class 'tvb_nest.nest_models.network.NESTNetwork'> create

2020-01-09 11:44:35,364 - DEBUG - tvb_multiscale.simulator_tvb.simulator - initial state has shape (2, 68, 1)
2020-01-09 11:44:35,368 - INFO - tvb_multiscale.simulator_tvb.simulator - Memory requirement census: simulation will need about 1.8 MB
2020-01-09 11:44:35,368 - INFO - tvb_multiscale.simulator_tvb.simulator - Memory requirement census: simulation will need about 1.8 MB
2020-01-09 11:44:35,370 - INFO - tvb_multiscale.simulator_tvb.simulator - Simulation runtime should be about 0.894 seconds
2020-01-09 11:44:35,370 - INFO - tvb_multiscale.simulator_tvb.simulator - Simulation runtime should be about 0.894 seconds
2020-01-09 11:44:35,372 - INFO - tvb_multiscale.simulator_tvb.simulator - Calculating storage requirement for ...
2020-01-09 11:44:35,372 - INFO - tvb_multiscale.simulator_tvb.simulator - Calculating storage requirement for ...
2020-01-09 11:44:35,374 - INFO - tvb_multiscale.simulator_tvb.simulator - Calculated storage requirement for simulation: 1224000 
2020-01-09 11:44

([(array([  0.1,   0.2,   0.3,   0.4,   0.5,   0.6,   0.7,   0.8,   0.9,
            1. ,   1.1,   1.2,   1.3,   1.4,   1.5,   1.6,   1.7,   1.8,
            1.9,   2. ,   2.1,   2.2,   2.3,   2.4,   2.5,   2.6,   2.7,
            2.8,   2.9,   3. ,   3.1,   3.2,   3.3,   3.4,   3.5,   3.6,
            3.7,   3.8,   3.9,   4. ,   4.1,   4.2,   4.3,   4.4,   4.5,
            4.6,   4.7,   4.8,   4.9,   5. ,   5.1,   5.2,   5.3,   5.4,
            5.5,   5.6,   5.7,   5.8,   5.9,   6. ,   6.1,   6.2,   6.3,
            6.4,   6.5,   6.6,   6.7,   6.8,   6.9,   7. ,   7.1,   7.2,
            7.3,   7.4,   7.5,   7.6,   7.7,   7.8,   7.9,   8. ,   8.1,
            8.2,   8.3,   8.4,   8.5,   8.6,   8.7,   8.8,   8.9,   9. ,
            9.1,   9.2,   9.3,   9.4,   9.5,   9.6,   9.7,   9.8,   9.9,
           10. ,  10.1,  10.2,  10.3,  10.4,  10.5,  10.6,  10.7,  10.8,
           10.9,  11. ,  11.1,  11.2,  11.3,  11.4,  11.5,  11.6,  11.7,
           11.8,  11.9,  12. ,  12.1,  12.2,  12.3,

2020-01-09 11:45:20,509 - DEBUG - matplotlib.pyplot - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [7]:
import nest
nest.Models()

('ac_generator',
 'aeif_cond_alpha',
 'aeif_cond_alpha_RK5',
 'aeif_cond_alpha_multisynapse',
 'aeif_cond_beta_multisynapse',
 'aeif_cond_exp',
 'aeif_psc_alpha',
 'aeif_psc_delta',
 'aeif_psc_delta_clopath',
 'aeif_psc_exp',
 'amat2_psc_exp',
 'bernoulli_synapse',
 'bernoulli_synapse_lbl',
 'clopath_synapse',
 'clopath_synapse_lbl',
 'cont_delay_synapse',
 'cont_delay_synapse_hpc',
 'cont_delay_synapse_lbl',
 'correlation_detector',
 'correlomatrix_detector',
 'correlospinmatrix_detector',
 'dc_generator',
 'diffusion_connection',
 'diffusion_connection_lbl',
 'erfc_neuron',
 'gamma_sup_generator',
 'gap_junction',
 'gap_junction_lbl',
 'gauss_rate_ipn',
 'gif_cond_exp',
 'gif_cond_exp_multisynapse',
 'gif_pop_psc_exp',
 'gif_psc_exp',
 'gif_psc_exp_multisynapse',
 'ginzburg_neuron',
 'hh_cond_beta_gap_traub',
 'hh_cond_exp_traub',
 'hh_psc_alpha',
 'hh_psc_alpha_clopath',
 'hh_psc_alpha_gap',
 'ht_neuron',
 'ht_synapse',
 'ht_synapse_hpc',
 'ht_synapse_lbl',
 'iaf_chs_2007',
 'iaf_ch

In [8]:
from pynestml.frontend.pynestml_frontend import to_nest, install_nest
import os
models_path = os.path.join(config.MODULES_DIR, "iaf_cond_deco2014")

In [9]:
install_nest(models_path, os.environ["NEST_INSTALL_DIR"])

In [10]:
import nest
nest.Models()

('ac_generator',
 'aeif_cond_alpha',
 'aeif_cond_alpha_RK5',
 'aeif_cond_alpha_multisynapse',
 'aeif_cond_beta_multisynapse',
 'aeif_cond_exp',
 'aeif_psc_alpha',
 'aeif_psc_delta',
 'aeif_psc_delta_clopath',
 'aeif_psc_exp',
 'amat2_psc_exp',
 'bernoulli_synapse',
 'bernoulli_synapse_lbl',
 'clopath_synapse',
 'clopath_synapse_lbl',
 'cont_delay_synapse',
 'cont_delay_synapse_hpc',
 'cont_delay_synapse_lbl',
 'correlation_detector',
 'correlomatrix_detector',
 'correlospinmatrix_detector',
 'dc_generator',
 'diffusion_connection',
 'diffusion_connection_lbl',
 'erfc_neuron',
 'gamma_sup_generator',
 'gap_junction',
 'gap_junction_lbl',
 'gauss_rate_ipn',
 'gif_cond_exp',
 'gif_cond_exp_multisynapse',
 'gif_pop_psc_exp',
 'gif_psc_exp',
 'gif_psc_exp_multisynapse',
 'ginzburg_neuron',
 'hh_cond_beta_gap_traub',
 'hh_cond_exp_traub',
 'hh_psc_alpha',
 'hh_psc_alpha_clopath',
 'hh_psc_alpha_gap',
 'ht_neuron',
 'ht_synapse',
 'ht_synapse_hpc',
 'ht_synapse_lbl',
 'iaf_chs_2007',
 'iaf_ch

In [11]:
nest.Install("iaf_cond_deco2014module")

In [12]:
nest.Models()

('ac_generator',
 'aeif_cond_alpha',
 'aeif_cond_alpha_RK5',
 'aeif_cond_alpha_multisynapse',
 'aeif_cond_beta_multisynapse',
 'aeif_cond_exp',
 'aeif_psc_alpha',
 'aeif_psc_delta',
 'aeif_psc_delta_clopath',
 'aeif_psc_exp',
 'amat2_psc_exp',
 'bernoulli_synapse',
 'bernoulli_synapse_lbl',
 'clopath_synapse',
 'clopath_synapse_lbl',
 'cont_delay_synapse',
 'cont_delay_synapse_hpc',
 'cont_delay_synapse_lbl',
 'correlation_detector',
 'correlomatrix_detector',
 'correlospinmatrix_detector',
 'dc_generator',
 'diffusion_connection',
 'diffusion_connection_lbl',
 'erfc_neuron',
 'gamma_sup_generator',
 'gap_junction',
 'gap_junction_lbl',
 'gauss_rate_ipn',
 'gif_cond_exp',
 'gif_cond_exp_multisynapse',
 'gif_pop_psc_exp',
 'gif_psc_exp',
 'gif_psc_exp_multisynapse',
 'ginzburg_neuron',
 'hh_cond_beta_gap_traub',
 'hh_cond_exp_traub',
 'hh_psc_alpha',
 'hh_psc_alpha_clopath',
 'hh_psc_alpha_gap',
 'ht_neuron',
 'ht_synapse',
 'ht_synapse_hpc',
 'ht_synapse_lbl',
 'iaf_chs_2007',
 'iaf_ch

In [13]:
n = nest.Create("iaf_cond_deco2014")

In [14]:
nest.GetStatus(n)

({'alpha': 0.5,
  'archiver_length': 0,
  'beta': 0.062,
  'beta_Ca': 0.001,
  'C_m': 500.0,
  'Ca': 0.0,
  'E_ex': 0.0,
  'E_in': -70.0,
  'E_L': -70.0,
  'element_type': <SLILiteral: neuron>,
  'frozen': False,
  'g_AMPA': 0.065,
  'g_GABA_A': 10.94,
  'g_L': 25.0,
  'g_NMDA': 0.2,
  'global_id': 479,
  'gsl_error_tol': 1e-06,
  'has_connections': False,
  'I_e': 0.0,
  'local': True,
  'local_id': 479,
  'model': <SLILiteral: iaf_cond_deco2014>,
  'n_receptors': 1,
  'node_uses_wfr': False,
  'parent': 0,
  'post_trace': 0.0,
  'recordables': (<SLILiteral: I_e>,
   <SLILiteral: V_m>,
   <SLILiteral: s_AMPA>,
   <SLILiteral: s_GABA>,
   <SLILiteral: x_NMDA>,
   <SLILiteral: s_NMDA>,
   <SLILiteral: I_L>,
   <SLILiteral: I_AMPA>,
   <SLILiteral: I_NMDA>,
   <SLILiteral: I_GABA>,
   <SLILiteral: spikes_exc>,
   <SLILiteral: spikes_inh>,
   <SLILiteral: s_AMPA_ext_0>,
   <SLILiteral: I_AMPA_ext_0>,
   <SLILiteral: spikes_exc_ext_0>),
  'supports_precise_spikes': False,
  'synaptic_eleme

In [15]:
import numpy as np
n1 = nest.Create("iaf_cond_deco2014", params={"w_E_ext": np.array([0.1, 0.1])})
n2 = nest.Create("iaf_cond_deco2014", params={"w_E_ext": np.array([0.1, 0.1])})

NESTErrors.BadProperty: ("BadProperty in SetDefaults_l_D: Setting status of model 'iaf_cond_deco2014': If the number of receptor ports have changed both arrays w_E_ext and s_AMPA_ext_max must be provided!", 'BadProperty', <SLILiteral: SetDefaults_l_D>, ": Setting status of model 'iaf_cond_deco2014': If the number of receptor ports have changed both arrays w_E_ext and s_AMPA_ext_max must be provided!")

In [ ]:
nest.Connect(n1, n1, syn_spec={"receptor_type": 0})
nest.Connect(n2, n2, syn_spec={"receptor_type": 0})
nest.Connect(n1, n2, syn_spec={"receptor_type": 1})
nest.Connect(n2, n1, syn_spec={"receptor_type": 2})

In [ ]:
m = nest.Create("multimeter", params={"record_from": ["s_AMPA_ext"]})
nest.Connect(m, n1 + n2)

In [ ]:
nest.Simulate(10)

In [ ]:
nest.GetStatus(m, "events")

In [ ]:
nest.GetConnections(m)

In [ ]:
import nest
m = nest.Create("multimeter", params={"record_from": ["s_AMPA_ext"]})

In [ ]:
import sys
sys.path